In [1]:
import sys 

In [ ]:
cd '/srv/share3/hagrawal9/project/habitat/habitat-api/'

In [ ]:
import gzip
import json
import os
import sys
from typing import Any, Dict, List, Optional, Type

import attr
import cv2
import git
import magnum as mn
import numpy as np

# %matplotlib inline
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image


import habitat
import habitat_sim
from habitat.config import Config
from habitat.core.registry import registry
from habitat_sim.utils import viz_utils as vut
from habitat.utils.visualizations import maps, fog_of_war
from habitat.utils.visualizations.utils import observations_to_image

from rearrangement.utils.planner import (
    compute_traversable_map,
    find_all_pair_shortest_path
)

In [ ]:
from habitat.datasets.rearrangement.rearrangement_dataset import RearrangementDatasetV0
from habitat.tasks.rearrangement.rearrangement_task import RearrangementEpisode

In [ ]:
repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
# %cd $dir_path
data_path = os.path.join(dir_path, "data")
output_directory = "data/tutorials/output/"  # @param {type:"string"}
output_path = os.path.join(dir_path, output_directory)

In [ ]:
config = habitat.get_config("configs/tasks/rearrangement_gibson.yaml")

In [ ]:
config.defrost()
config.TASK.MEASUREMENTS = ['OBJECT_TO_GOAL_DISTANCE', 'AGENT_TO_OBJECT_DISTANCE', 'TOP_DOWN_MAP']
config.freeze()

In [8]:
def make_video_cv2(
    observations, cross_hair=None, prefix="", open_vid=True, fps=60
):
    sensor_keys = list(observations[0])
    videodims = observations[0][sensor_keys[0]].shape
    videodims = (videodims[1], videodims[0])  # flip to w,h order
    print(videodims)
    video_file = output_path + prefix + ".mp4"
    print("Encoding the video: %s " % video_file)
    writer = vut.get_fast_video_writer(video_file, fps=fps)
    for ob in observations:
        # If in RGB/RGBA format, remove the alpha channel
        rgb_im_1st_person = cv2.cvtColor(ob["rgb"], cv2.COLOR_RGBA2RGB)
        if cross_hair is not None:
            rgb_im_1st_person[
                cross_hair[0] - 2 : cross_hair[0] + 2,
                cross_hair[1] - 2 : cross_hair[1] + 2,
            ] = [255, 0, 0]

        if rgb_im_1st_person.shape[:2] != videodims:
            rgb_im_1st_person = cv2.resize(
                rgb_im_1st_person, videodims, interpolation=cv2.INTER_AREA
            )
        # write the 1st person observation to video
        writer.append_data(rgb_im_1st_person)
    writer.close()

    if open_vid:
        print("Displaying video")
        vut.display_video(video_file)


In [9]:
def print_info(obs, metrics):
    print(
        "Gripped Object: {}, Distance To Object: {}, Distance To Goal: {}".format(
            obs["gripped_object_id"],
            metrics["agent_to_object_distance"],
            metrics["object_to_goal_distance"],
        )
    )

In [10]:
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--no-display", dest="display", action="store_false")
    parser.add_argument(
        "--no-make-video", dest="make_video", action="store_false"
    )
    parser.set_defaults(show_video=True, make_video=True)
    args, _ = parser.parse_known_args()
    show_video = args.display
    display = args.display
    make_video = args.make_video
else:
    show_video = False
    make_video = False
    display = False

In [11]:
try:
    env.close()
except:
    pass
env = habitat.Env(config)
fog_of_war_mask = None

2020-10-07 23:51:36,575 Initializing dataset RearrangementDataset-v0
2020-10-07 23:51:36,580 initializing sim RearrangementSim-v0
I1007 23:51:41.796565 23087 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Barboursville.navmesh
I1007 23:51:41.799084 23087 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.
2020-10-07 23:51:41,959 Initializing task RearrangementTask-v0


In [14]:
obs = env.reset()
metrics = env.get_metrics()

rotating agent!


In [19]:
def get_top_down_map(env, fog_of_war_mask=None, should_draw_fow=True, should_draw_agent=False, should_draw_object=False):
    
    env._sim.recompute_navmesh(env._sim.pathfinder, env._sim.navmesh_settings, False)

    top_down_map = maps.get_topdown_map(
        env._sim.pathfinder,
        env._sim.get_agent(0).state.position[1],
        1024
    )

    agent_position = env._sim.get_agent_state().position
    a_y, a_x = maps.to_grid(
        agent_position[2],
        agent_position[0],
        top_down_map.shape[0:2],
        sim=env._sim,
    )

    episode = env.current_episode
    object_positions = [obj.position for obj in episode.objects]
    goal_positions = [obj.position for obj in episode.goals]

    grid_object_positions = []
    grid_goal_positions = []

    for i, obj_pos in enumerate(object_positions):
        tdm_pos = maps.to_grid(
            obj_pos[2],
            obj_pos[0],
            top_down_map.shape[0:2],
            sim=env._sim,
        )
        grid_object_positions.append(tdm_pos)

    # draw the objectgoal positions.
    for i, goal_pos in enumerate(goal_positions):
        tdm_pos = maps.to_grid(
            goal_pos[2],
            goal_pos[0],
            top_down_map.shape[0:2],
            sim=env._sim,
        )

        grid_goal_positions.append(tdm_pos)
    
    agent_rotation = metrics["top_down_map"]["agent_angle"]
    # print(agent_rotation)
    
    if should_draw_fow:
        if fog_of_war_mask is None:
            fog_of_war_mask = np.zeros_like(top_down_map)
        
        fog_of_war_mask = fog_of_war.reveal_fog_of_war(
            np.ones(top_down_map.shape),
            fog_of_war_mask,
            np.array([a_y, a_x]),
            agent_rotation,
            fov = 90,
            max_line_len = 500
        )
    top_down_map = maps.colorize_topdown_map(top_down_map)
    
    if should_draw_agent:
        top_down_map = maps.draw_agent(
            image=top_down_map,
            agent_center_coord=[a_y, a_x],
            agent_rotation=agent_rotation,
            agent_radius_px=min(top_down_map.shape[0:2]) / 32,
        )
    
    if should_draw_object:
        top_down_map = maps.draw_object_info(top_down_map, grid_object_positions, grid_goal_positions)
    
    return top_down_map, fog_of_war_mask

In [ ]:
for i in range(10):
    obs = env.step(2)

for i in range(6):
    obs = env.step(1)

for i in range(5):
    obs = env.step(2)

for i in range(5):
    obs = env.step(1)
    
for i in range(10):
    obs = env.step(2)
    
metrics = env.get_metrics()

plt.imshow(obs['depth'].reshape((480, 640)), cmap="gray")
plt.show()